In [1]:
import pandas as pd
import os
import requests
import tweepy
import json
import re

### 收集数据
#### WeRateDog的twitter档案

In [2]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')


#### twitter图像预测数据

In [3]:
# download the file of image predictions from the URL

url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
response = requests.get(url)

with open(url.split('/')[-1], mode='wb') as file:
        file.write(response.content)
        

In [4]:
# upload the date from the file
image_predictions = pd.read_csv(url.split('/')[-1], sep='\t')


#### twitter附加数据

In [ ]:
# get twitter API
consumer_key = 'YOUR CONSUMER KEY'
consumer_secret = 'YOUR CONSUMER SECRET'
access_token = 'YOUR ACCESS TOKEN'
access_secret = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [ ]:
# find the twitter content
for tweet_id in twitter_archive['tweet_id']:
    try:
        # download the content of the specific id
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        # append data into txt
        with open('tweet_json.txt', 'w') as f:
            f.write(json.dump(tweet, f, ensure_ascii=False))
    # not best practice to catch all exceptions        
    except Exception as e:
        print('%: The content of % has been deleted'.format(e, tweet_id))
        

In [7]:
# read txt data to build a list of dictionary
json_list = []

with open('tweet_json.txt', 'r') as f:
    for line in f.readlines():
        df = json.loads(line)

        json_list.append(df)


In [8]:
# create DataFrame from list of dictionary
df_list = []
for item in json_list:
    tweet_ID = item['id']
    retweet_count = item['retweet_count']
    favorite_count = item['favorite_count']
    
    df_list.append({'tweet_id': tweet_ID,
                   'retweet_count': retweet_count,
                   'favorite_count': favorite_count})
    
tweet_extra_data = pd.DataFrame(df_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])


### 数据评估

通过目测评估和编程评估的方式对数据进行质量及整洁度的评估

#### 质量 quality  

*twitter_archive table*
* 'tweet_id' is an int not a string
* The columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' have too much missing data 
* The dataset doesn't need retweet contents
* 'timestamp' is a string not a datatype and contains '+0000'
* 'source' includes useless infomation such like http link
* 'expanded_urls' has null value or duplicated urls in a row
* Min and max numbers of the 'rating_numerator' and 'rating_denominator' are not correct
* Many None or a/an in 'name'
* 'doggo', 'floofer', 'pupper', 'puppo' contain too many None
* Delete the data after 8/1/2017

*image_predictions table*
1. 'tweet_id' and 'img_num' is an int not a string
2. The duplicates in 'jpg_url' column


#### 整洁度 tidiness
* Dog stage is in four different columns (doggo, floofer, pupper, puppo)
* Three tables can be merged


In [13]:
twitter_archive.sample(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2096,670755717859713024,NaN,NaN,2015-11-29 00:06:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Gin &amp; Tonic. They're having a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670755717...,9,10,Gin,None,None,None,None
1731,679844490799091713,NaN,NaN,2015-12-24 02:02:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Willie. He's floating away and needs y...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/679844490...,10,10,Willie,None,None,None,None
28,886680336477933568,NaN,NaN,2017-07-16 20:14:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Derek. He's late for a dog meeting. 13...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/886680336...,13,10,Derek,None,None,None,None
1218,714957620017307648,NaN,NaN,2016-03-29 23:29:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Curtis. He's an Albino Haberdasher. Te...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714957620...,10,10,Curtis,None,None,None,None
2132,670079681849372674,NaN,NaN,2015-11-27 03:20:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Fabio. He's a wonderful pup. Can't stay a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670079681...,10,10,Fabio,None,None,None,None
1232,713175907180089344,NaN,NaN,2016-03-25 01:29:21 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Opie and Clarkus. Clarkus fell as...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713175907...,10,10,Opie,None,None,None,None
2097,670733412878163972,NaN,NaN,2015-11-28 22:38:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jerry. He's a great listener. Low main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670733412...,8,10,Jerry,None,None,None,None
1402,699423671849451520,NaN,NaN,2016-02-16 02:42:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Don't ever talk to me or my son again."" ...bo...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/699423671...,10,10,None,None,None,None,None
2197,668826086256599040,NaN,NaN,2015-11-23 16:18:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Banditt. He is a brown LaBeouf retriev...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668826086...,10,10,Banditt,None,None,None,None
641,793226087023144960,NaN,NaN,2016-10-31 23:00:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Juno. She spooked me up real good, but...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/793226087...,11,10,Juno,None,None,None,None


In [14]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [ ]:
twitter_archive.describe()

In [ ]:
twitter_archive.sample(5)

In [ ]:
twitter_archive['name'].value_counts()

In [ ]:
image_predictions.info()

In [ ]:
image_predictions.describe()

In [ ]:
image_predictions.sample(10)

In [ ]:
image_predictions[image_predictions.jpg_url.duplicated()]

In [ ]:
tweet_extra_data.info()

In [ ]:
tweet_extra_data.describe()

In [ ]:
tweet_extra_data.sample(10)

### 数据清理


In [29]:
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
tweet_extra_data_clean = tweet_extra_data.copy()

#### 缺失数据

`twitter_archive`: 
* The columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' have too much missing data. 
* The dataset doesn't need retweet contents

###### 定义

* Delete the columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'
* Delete the rows including the 'in_reply_to_status_id' and 'retweeted_status_user_id'

###### 代码

In [30]:
# Delete the rows including the 'in_reply_to_status_id' and 'retweeted_status_user_id'

twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['in_reply_to_status_id'].isnull()]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['retweeted_status_user_id'].isnull()]

In [31]:
# Delete the columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'

twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 
                            'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

###### 测试

In [ ]:
twitter_archive_clean.sample(5)

##### 整洁度


`twitter_archive`:
* Dog stage is in four different columns (doggo, floofer, pupper, puppo)

###### 定义

* Find the stage from the text
* Delete the 'doggo', 'floofer', 'pupper', 'puppo' columns

###### 代码

In [32]:
# Find the stage from the text
twitter_archive_clean['stage'] = twitter_archive_clean.text.str.lower().str.findall('(doggo|floofer|pupper|puppo)')

# Transfer 'stage' list to string
twitter_archive_clean['stage'] = twitter_archive_clean.stage.str.join('')

# Delete the 'doggo', 'floofer', 'pupper', 'puppo' columns
twitter_archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

##### 测试

In [ ]:
twitter_archive_clean.sample(5)

##### merge three tables at the final step

#### 质量

#### `twitter_archive` & `image_predictions`:
* 'tweet_id' is an int not a string

###### 定义

* Cover the 'tweet_id' column's data from float to a string

###### 代码

In [33]:
# Cover the 'tweet_id' column's data from float to a string
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
image_predictions_clean['tweet_id'] = image_predictions_clean['tweet_id'].astype(str)


##### 测试

In [ ]:
twitter_archive_clean.info()

In [ ]:
image_predictions_clean.info()


#### `twitter_archive`:
* 'timestamp' is a string not a datatype


##### 定义
* Slice the 'timestamp' without +0000
* Cover the 'timestamp' column's data from string to datatime

##### 代码

In [34]:
# Slice the 'timestamp' without +0000
# Cover the 'timestamp' column's data from string to datatime
twitter_archive_clean.timestamp = pd.to_datetime(twitter_archive_clean.timestamp[:-6])

##### 测试

In [ ]:
twitter_archive_clean.head()

In [ ]:
twitter_archive_clean.info()

#### `twitter_archive`:
* 'source' includes useless infomation such like http link

##### 定义
* Just cut out the info of where the source comes, such like Twitter for iPhone, Twitter Web Client and Vine - Make a Scene

##### 代码

In [35]:
twitter_archive_clean['source'] = twitter_archive_clean.source

##### 测试

In [ ]:
twitter_archive_clean.sample()

#### `twitter_archive`:
* 'expanded_urls' has null value or duplicated urls in a row

##### 定义
* Keep the rows which is not null using `notnull()`

##### 代码

In [ ]:
# Keep the rows which is not null using notnull()

twitter_archive_clean[twitter_archive_clean['expanded_urls'].isnull()]

In [36]:
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['expanded_urls'].notnull()]

#####  测试

In [ ]:
twitter_archive_clean['expanded_urls'].value_counts()

In [ ]:
twitter_archive_clean.sample(5)

#### `twitter_archive`:
* Min and max numbers of the 'rating_numerator' and 'rating_denominator' are not correct

##### 定义
* Reselect the rating numbers from the texts
* Check the outlier
* If the denominator is not 10, double check the text or delete the row

##### 代码

In [38]:
# Reselect the rating numbers from the texts

twitter_archive_clean['rating'] = twitter_archive_clean.text.str.extract('(\d+\/\d+)',expand=True)

twitter_archive_clean['rating'].value_counts()

12/10      485
10/10      434
11/10      413
13/10      287
9/10       152
8/10        98
7/10        51
14/10       38
5/10        34
6/10        32
3/10        19
4/10        15
2/10         9
1/10         4
26/10        1
88/80        1
9/11         1
84/70        1
27/10        1
204/170      1
165/150      1
75/10        1
60/50        1
0/10         1
1/2          1
99/90        1
1776/10      1
80/80        1
121/110      1
7/11         1
44/40        1
24/7         1
420/10       1
50/50        1
45/50        1
144/120      1
4/20         1
Name: rating, dtype: int64

In [40]:
# list all rows contains outliers

twitter_archive_clean[
    twitter_archive_clean['rating'].isin([
        '26/10','88/80','9/11','84/70','27/10','204/170','165/150', '75/10',
    '60/50', '0/10', '1/2', '99/90', '1776/10', '80/80', '121/110', '7/11',
    '44/40', '24/7', '420/10', '50/50', '45/50', '144/120', '4/20'
    ])][['text', 'rating', 'rating_numerator', 'rating_denominator']]

,text,rating,rating_numerator,rating_denominator
315,When you're so blinded by your systematic plag...,0/10,0,10
433,The floofs have been released I repeat the flo...,84/70,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24/7,24,7
695,"This is Logan, the Chow who lived. He solemnly...",75/10,75,10
763,This is Sophie. She's a Jubilant Bush Pupper. ...,27/10,27,10
902,Why does this never happen at my front door......,165/150,165,150
979,This is Atticus. He's quite simply America af....,1776/10,1776,10
1068,"After so many requests, this is Bretagne. She ...",9/11,9,11
1120,Say hello to this unbelievably well behaved sq...,204/170,204,170
1165,Happy 4/20 from the squad! 13/10 for all https...,4/20,4,20
